In [29]:
import json
from dateutil import parser


In [ ]:
# https://www.essl.org/cms/wp-content/uploads/ESSL-rep-2020-11-1.pdf?utm_source=chatgpt.com
def map_to_category_and_severity(type_event):
    mapping = {
        "AVALANCHE": ("None", 3),
        "DEVIL": ("None", 3),
        "FUNNEL": ("None", 3),
        "GUSTNADO": ("None", 3),
        "HAIL": ("HAIL", 3),
        "ICE": ("ICE", 3),
        "LIGHTNING": ("LIGHTNING", 3),
        "PRECIP": ("RAIN", 3),
        "SNOW": ("SNOW", 3),
        "TORNADO": ("TORNADO", 3),
        "WIND": ("WIND", 3),
    }
    return mapping.get(type_event, ("Unknown", 0))

def map_link_id(link_id):
    return int(link_id) if link_id and link_id.isdigit() else None

def map_time(time_str):
    return int(parser.isoparse(time_str).timestamp() * 1000)

def map_place(record_item):
    return f'{record_item.get("COUNTRY")}, {record_item.get("PLACE")}'


In [34]:
def map_input_to_output(input_record):
    category, severity = map_to_category_and_severity(input_record.get("TYPE_EVENT"))
    return {
        "type": "Feature",
        "properties": {
            "meldungId": map_link_id(input_record.get("LINK_ID")),
            "crowdDeviceId": None,
            "place": map_place(input_record),
            "timestamp": map_time(input_record.get("TIME_EVENT")),
            "submissionTimestamp": map_time(input_record.get("TIME_CREATION")),
            "category": category,
            "auspraegung": severity,
            "zusatzAttribute": [],
            "comment": input_record.get("EVENT_DESCRIPTION"),
            "imageOriginalUrl": None,
            "imageUrl": None,
            "imageThumbnailUrl": None,
            "numberOfSubmissions": input_record.get("NO_REVISION"),
            "approvalRate": 0.0,
            "qualityCheckPassed": not input_record.get("DELETED", False),
            "missingValidationData": False,
            "qualityCheckMsg": None,
            "timesReportedForImage": 0,
            "timesReportedForWeather": 0,
            "userType": None,
            "sourceVersion": None,
            "source": input_record.get("LINK_ORG"),
            "imageLabels": {
                "labels": [],
                "nsfwLabels": []
            }
        },
        "geometry": { 
            "type": "Point",    
             "coordinates": [
                input_record.get("LONGITUDE"),
                input_record.get("LATITUDE"),
            ]
        }
    }


In [35]:
def read_input_json(input_path):
    with open(input_path, 'r') as f:
        return json.load(f)

def transform_all(input_data):
    return [map_input_to_output(item) for item in input_data]

def write_geojson_output(output_data, output_path='output.json'):
    output = {"type":"FeatureCollection","features": output_data}
    with open(output_path, 'w') as f:
        json.dump(output, f, indent=4)

In [36]:
input_path = 'sample.json'
data = read_input_json(input_path)

output_data = transform_all(data)
write_geojson_output(output_data)